In [ ]:
# @title 1. Setup (local/Colab agnostic)
import os, sys, subprocess
from pathlib import Path

# Configure project directory (default: ./FPL model relative to this notebook)
PROJECT_PATH = Path(os.getenv("FPL_PROJECT_PATH", Path.cwd() / "FPL model")).resolve()
PROJECT_PATH.mkdir(parents=True, exist_ok=True)
os.chdir(PROJECT_PATH)
print(f"📂 Working in: {PROJECT_PATH}")

# External historical data repo path (override with FPL_HISTORY_REPO env var)
HISTORY_REPO = os.getenv("FPL_HISTORY_REPO", "Fantasy-Premier-League")

# Install dependencies if missing (safe to rerun)
required = {
    "xgboost": "xgboost",
    "scikit-learn": "sklearn",
    "joblib": "joblib",
    "pandas": "pandas",
    "requests": "requests",
    "pulp": "pulp",
    "soccerdata": "soccerdata",
}

missing = []
for pip_name, import_name in required.items():
    try:
        __import__(import_name)
    except ImportError:
        missing.append(pip_name)

if missing:
    print(f"📦 Installing: {', '.join(missing)}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])
else:
    print("✅ All required packages are available.")



In [ ]:
# @title 2A. Create Fetch Script
%%writefile fetch_live_data.py
import requests
import pandas as pd
import os

print("--- FETCHING LIVE FPL DATA ---")
base_url = 'https://fantasy.premierleague.com/api/'
r = requests.get(base_url + 'bootstrap-static/')
json_data = r.json()

players_df = pd.DataFrame(json_data['elements'])
teams_df = pd.DataFrame(json_data['teams'])
events_df = pd.DataFrame(json_data['events'])

cols_to_keep = ['id', 'web_name', 'first_name', 'second_name', 'element_type', 'team', 'now_cost', 'total_points', 'form', 'ep_next', 'status', 'news']
clean_players = players_df[cols_to_keep]

team_map = pd.Series(teams_df.name.values, index=teams_df.id).to_dict()
clean_players['team_name'] = clean_players['team'].map(team_map)

os.makedirs('data/current_season', exist_ok=True)
clean_players.to_csv('data/current_season/players_raw.csv', index=False)
events_df.to_csv('data/current_season/gameweeks.csv', index=False)
print("✅ Live data saved.")

In [ ]:
# @title 2B. Create Merge Script (V2)
%%writefile merge_data.py
import pandas as pd
import glob
import os
from pathlib import Path

print("--- MERGING HISTORICAL & LIVE DATA (V2) ---")

HISTORY_REPO = os.getenv("FPL_HISTORY_REPO", "Fantasy-Premier-League")
repo_glob = str(Path(HISTORY_REPO) / "data" / "202*-2*" / "gws" / "merged_gw.csv")

try:
    current_players = pd.read_csv('data/current_season/players_raw.csv')
except FileNotFoundError:
    print("❌ Error: Live data not found. Run fetch_live_data.py first.")
    exit()

history_files = glob.glob(repo_glob)
history_list = []
for f in history_files:
    df = pd.read_csv(f, encoding='latin1', low_memory=False)
    season_label = f.split('/')[-3]
    df['season'] = season_label
    history_list.append(df)

full_history = pd.concat(history_list)
full_history['kickoff_time'] = pd.to_datetime(full_history['kickoff_time'])
full_history = full_history.sort_values(['element', 'kickoff_time'])

print("⚙️ Calculating historical performance against opponents...")
full_history['avg_points_vs_opponent'] = full_history.groupby(['element', 'opponent_team'])['total_points'].transform(
    lambda x: x.expanding().mean().shift(1)
)

global_avg = full_history.groupby('element')['total_points'].transform('mean')
full_history['avg_points_vs_opponent'] = full_history['avg_points_vs_opponent'].fillna(global_avg)

full_history.to_csv('data/training_dataset.csv', index=False)
print("✅ Saved V2 Master Dataset with Opponent History.")



In [ ]:
# @title 3. 🔄 WEEKLY UPDATE (Run this every time!)
import os
from pathlib import Path

print("🔄 Starting Weekly Update Process...")

repo_path = Path(os.getenv("FPL_HISTORY_REPO", "Fantasy-Premier-League"))
repo_str = str(repo_path)

if repo_path.exists():
    print("⬇️ Updating Vaastav's Historical Data...")
    !cd "$repo_str" && git pull
else:
    print("⚠️ Repo not found. Cloning now...")
    !git clone https://github.com/vaastav/Fantasy-Premier-League.git "$repo_str"

print("⬇️ Fetching Live API Data...")
!python fetch_live_data.py

print("⚙️ Merging Datasets...")
!python merge_data.py

print("
✅ UPDATE COMPLETE. You are ready to train.")



In [ ]:
# @title 4. Train XGBoost Model (V2: Learned Opponent History)
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import joblib

TRAINING_FILE = 'data/training_dataset.csv'
MODEL_FILE = 'fpl_model_v2.pkl'

print("1️⃣ Loading V2 Dataset...")
df = pd.read_csv(TRAINING_FILE, low_memory=False)
df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])
df = df.sort_values(['element', 'kickoff_time'])

print("2️⃣ Re-Engineering Features...")
df['last_points'] = df.groupby('element')['total_points'].shift(1)
df['form_last_3'] = df.groupby('element')['total_points'].transform(lambda x: x.rolling(window=3).mean().shift(1))
df['minutes_last_3'] = df.groupby('element')['minutes'].transform(lambda x: x.rolling(window=3).mean().shift(1))
df['was_home_last'] = df.groupby('element')['was_home'].shift(1)

if 'difficulty' in df.columns:
    df['opponent_difficulty'] = df['difficulty']
else:
    df['opponent_difficulty'] = 3

# NEW: Ensure the new feature exists (if not, fill with 0 to prevent crash)
if 'avg_points_vs_opponent' not in df.columns:
    df['avg_points_vs_opponent'] = 0

df = df.dropna(subset=['last_points', 'form_last_3', 'minutes_last_3'])

# --- NEW FEATURE ADDED HERE ---
features = ['value', 'was_home', 'opponent_difficulty', 'last_points', 'form_last_3', 'minutes_last_3', 'avg_points_vs_opponent']
target = 'total_points'

split_point = int(len(df) * 0.85)
X_train = df[features].iloc[:split_point]
y_train = df[target].iloc[:split_point]
X_test = df[features].iloc[split_point:]
y_test = df[target].iloc[split_point:]

print(f"3️⃣ Training V2 Model on {len(X_train)} rows...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=150,
    learning_rate=0.05,
    max_depth=5,
    n_jobs=-1
)

model.fit(X_train, y_train)

predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"🎯 V2 Model MAE: {mae:.3f}")

joblib.dump(model, MODEL_FILE)
print(f"✅ V2 Model saved as '{MODEL_FILE}'")

In [ ]:
# @title 5. 🔮 Generate Predictions (V2.1: Fixed - Saves All Columns)
import pandas as pd
import numpy as np
import joblib
import requests

print("Loading V2 model...")
try:
    model = joblib.load('fpl_model_v2.pkl')
except FileNotFoundError:
    print("❌ Error: Model not found. Did you run Cell 4?")
    raise

history_df = pd.read_csv('data/training_dataset.csv', low_memory=False)

# Live Data
r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
live_data = r.json()
live_players = pd.DataFrame(live_data['elements'])
teams_df = pd.DataFrame(live_data['teams'])
types_df = pd.DataFrame(live_data['element_types'])
team_map = pd.Series(teams_df.name.values, index=teams_df.id).to_dict()
pos_map = pd.Series(types_df.singular_name_short.values, index=types_df.id).to_dict()

# Fixtures
fixtures = requests.get('https://fantasy.premierleague.com/api/fixtures/?future=1').json()
fixtures_df = pd.DataFrame(fixtures)

next_matches = {}
for team_id in range(1, 21):
    next_match = fixtures_df[
        ((fixtures_df['team_h'] == team_id) | (fixtures_df['team_a'] == team_id)) &
        (fixtures_df['finished'] == False)
    ].iloc[0]

    if next_match['team_h'] == team_id:
        next_matches[team_id] = {'difficulty': next_match['team_h_difficulty'], 'was_home': True, 'opponent': next_match['team_a'], 'opp_name': team_map.get(next_match['team_a'])}
    else:
        next_matches[team_id] = {'difficulty': next_match['team_a_difficulty'], 'was_home': False, 'opponent': next_match['team_h'], 'opp_name': team_map.get(next_match['team_h'])}

print("Building V2 input data...")
predict_list = []

for index, player in live_players.iterrows():
    if player['status'] == 'u': continue

    p_id = player['id']
    team_id = player['team']

    player_history = history_df[history_df['element'] == p_id]

    if not player_history.empty:
        last_match = player_history.iloc[-1]

        # --- CALCULATE HISTORY VS NEXT OPPONENT ---
        next_opponent_id = next_matches[team_id]['opponent']
        vs_opp_history = player_history[player_history['opponent_team'] == next_opponent_id]

        if not vs_opp_history.empty:
            avg_vs_opp = vs_opp_history['total_points'].mean()
        else:
            avg_vs_opp = player_history['total_points'].mean()

        row = {
            'element': p_id,
            'full_name': f"{player['first_name']} {player['second_name']}",
            'web_name': player['web_name'],
            'position': pos_map.get(player['element_type']),
            'team_name': team_map.get(team_id),
            'next_opponent': next_matches[team_id]['opp_name'],
            'now_cost': player['now_cost'],

            # RAW COLUMNS (Required for Solver)
            'element_type': player['element_type'],
            'team': player['team'],

            # Features
            'value': player['now_cost'],
            'was_home': next_matches[team_id]['was_home'],
            'opponent_difficulty': next_matches[team_id]['difficulty'],
            'last_points': last_match['total_points'],
            'form_last_3': last_match.get('form_last_3', 0),
            'minutes_last_3': last_match.get('minutes_last_3', 0),
            'avg_points_vs_opponent': avg_vs_opp
        }
        predict_list.append(row)

input_df = pd.DataFrame(predict_list)
input_df = input_df.fillna(0)

features = ['value', 'was_home', 'opponent_difficulty', 'last_points', 'form_last_3', 'minutes_last_3', 'avg_points_vs_opponent']
input_df['predicted_points'] = model.predict(input_df[features])

# --- SAVE ALL COLUMNS (The Fix) ---
# We save everything so Cell 6 and 7 can read IDs and Names
input_df.to_csv('predictions.csv', index=False)

print("\n🔝 TOP 10 PREDICTED SCORERS (V2):")
print(input_df[['full_name', 'position', 'team_name', 'next_opponent', 'predicted_points']].head(10).to_string(index=False))

In [ ]:
# @title 6. 🧠 Optimize Squad (V2: Wildcard Solver)
!pip install pulp > /dev/null
import pulp
import pandas as pd

# --- CONFIGURATION ---
BUDGET = 1000  # 1000 = £100.0m
try:
    predictions = pd.read_csv('predictions.csv')
except FileNotFoundError:
    print("❌ Error: 'predictions.csv' not found. Please run Cell 5 first.")
    exit()

print(f"🤖 Optimizing squad for Budget £{BUDGET/10}m...")

# 1. Setup Data for Solver
# We create dictionaries mapping Index -> Value
players = predictions.index.tolist()
costs = predictions['now_cost'].to_dict()
points = predictions['predicted_points'].to_dict()
# Ensure element_type is integer
predictions['element_type'] = predictions['element_type'].astype(int)
positions = predictions['element_type'].to_dict()
teams = predictions['team'].to_dict()
# We use full_name for display if available, else web_name
names = predictions['full_name'].to_dict() if 'full_name' in predictions.columns else predictions['web_name'].to_dict()

# 2. Define the Linear Programming Problem
prob = pulp.LpProblem("FPL_Squad_Selection", pulp.LpMaximize)

# Decision Variable: Should we pick player 'i'? (1=Yes, 0=No)
x = pulp.LpVariable.dicts("player", players, 0, 1, pulp.LpBinary)

# OBJECTIVE: Maximize Total Predicted Points
prob += pulp.lpSum([points[i] * x[i] for i in players]), "Total_Points"

# CONSTRAINTS:
# Total Cost <= Budget
prob += pulp.lpSum([costs[i] * x[i] for i in players]) <= BUDGET, "Total_Cost"

# Total Players = 15
prob += pulp.lpSum([x[i] for i in players]) == 15, "Squad_Size"

# Position Limits (1=GK, 2=DEF, 3=MID, 4=FWD)
prob += pulp.lpSum([x[i] for i in players if positions[i] == 1]) == 2, "GK_Count"
prob += pulp.lpSum([x[i] for i in players if positions[i] == 2]) == 5, "Def_Count"
prob += pulp.lpSum([x[i] for i in players if positions[i] == 3]) == 5, "Mid_Count"
prob += pulp.lpSum([x[i] for i in players if positions[i] == 4]) == 3, "Fwd_Count"

# Max 3 Players per Team
# We get unique team IDs from the data to be safe
unique_teams = predictions['team'].unique()
for team_id in unique_teams:
    prob += pulp.lpSum([x[i] for i in players if teams[i] == team_id]) <= 3, f"Team_{team_id}_Limit"

# 3. Solve
status = prob.solve(pulp.PULP_CBC_CMD(msg=0))

if status != 1:
    print("❌ Error: Solution not found. Check if Budget is too low.")
else:
    # --- DISPLAY RESULTS (V2 Style) ---
    selected_indices = [i for i in players if x[i].varValue == 1]
    my_team = predictions.loc[selected_indices].copy()

    # Sort by Position (GK, DEF, MID, FWD) then Points
    my_team = my_team.sort_values(['element_type', 'predicted_points'], ascending=[True, False])

    total_score = my_team['predicted_points'].sum()
    total_cost = my_team['now_cost'].sum()

    print(f"\n✅ OPTIMAL SQUAD FOUND!")
    print(f"💰 Total Cost: £{total_cost/10}m")
    print(f"🔮 Projected Points: {total_score:.2f}")
    print("-" * 85)
    print(f"{'Pos':<5} {'Player':<25} {'Team':<15} {'Opponent':<15} {'Price':<8} {'Pts':<5}")
    print("-" * 85)

    pos_map = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}
    for _, row in my_team.iterrows():
        p_name = row['full_name'] if 'full_name' in row else row['web_name']
        t_name = row['team_name'] if 'team_name' in row else str(row['team'])
        opp_name = row['next_opponent'] if 'next_opponent' in row else "-"

        print(f"{pos_map[row['element_type']]:<5} {p_name:<25} {t_name:<15} {opp_name:<15} £{row['now_cost']/10:<7} {row['predicted_points']:.1f}")

In [ ]:
# @title 7. 🔄 The Transfer Recommender (V5: Conservative Strategy)
import pandas as pd

# --- 1. USER SETTINGS ---
# Your Team (Use Full Names if needed to avoid ambiguity)
my_current_team = [
    'Raya', 'Donnarumma',
    'Gabriel', 'Virgil', 'Mukiele', 'Hall', 'Andersen',
    'L.Miley', 'Anderson', 'Thiago', 'Garner', 'Foden',
    'Haaland', 'Harry Wilson', 'Woltemade'
]

BANK = 3.1           # Money in bank
FREE_TRANSFERS = 1   # How many transfers do you have? (1 or 2)
TRANSFER_THRESHOLD = 3.0 # Min points gain required to justify a transfer

# ----------------------------------------
print(f"🧐 Analyzing conservative transfers (Threshold: +{TRANSFER_THRESHOLD} pts)...")

try:
    preds = pd.read_csv('predictions.csv')
except FileNotFoundError:
    print("❌ Error: 'predictions.csv' not found. Please run Cell 5 first.")
    raise

# --- SMART IDENTITY CHECK ---
found_players = []
print("\n📝 Verifying your squad:")

for name in my_current_team:
    match = preds[preds['web_name'] == name]
    if len(match) == 0: match = preds[preds['full_name'] == name]

    if len(match) == 0:
        print(f"   ❌ NOT FOUND: '{name}'")
    elif len(match) > 1:
        # Try exact match on full_name
        exact = match[match['full_name'] == name]
        if len(exact) == 1:
             found_players.append(exact.iloc[0])
             print(f"   ✅ Found: {name:<15} -> {exact.iloc[0]['full_name']}")
        else:
            found_players.append(match.iloc[0]) # Default to first
            print(f"   ⚠️ AMBIGUOUS: '{name}' -> Using {match.iloc[0]['full_name']}")
    else:
        found_players.append(match.iloc[0])
        print(f"   ✅ Found: {name:<15} -> {match.iloc[0]['full_name']}")

current_stats = pd.DataFrame(found_players)
if current_stats.empty: raise ValueError("Team list is empty.")

current_total = current_stats['predicted_points'].sum()
print(f"\n📊 Current Team Projection: {current_total:.2f} pts")

# Identify Weakest Links
current_stats = current_stats.sort_values('predicted_points')

# --- THE SWAP LOGIC ---
best_transfer = None
max_gain = 0

for index, seller in current_stats.iterrows():
    seller_pred = seller['predicted_points']
    budget_available = seller['now_cost'] + (BANK * 10)

    # Find replacements
    options = preds[
        (preds['element_type'] == seller['element_type']) &
        (preds['now_cost'] <= budget_available) &
        (~preds['web_name'].isin(my_current_team))
    ]

    if not options.empty:
        best_option = options.sort_values('predicted_points', ascending=False).iloc[0]
        gain = best_option['predicted_points'] - seller_pred

        if gain > max_gain:
            max_gain = gain
            buy_name = best_option['full_name'] if 'full_name' in best_option else best_option['web_name']

            best_transfer = {
                'Out': seller['web_name'],
                'In': buy_name,
                'Gain': gain,
                'Cost_Diff': (best_option['now_cost'] - seller['now_cost']) / 10,
                'New_Points': best_option['predicted_points']
            }

# --- DECISION ENGINE ---
print("-" * 60)

if best_transfer and max_gain >= TRANSFER_THRESHOLD:
    print(f"🚨 TRANSFER RECOMMENDED (Gain > {TRANSFER_THRESHOLD}):")
    print(f"   ❌ SELL: {best_transfer['Out']}")
    print(f"   ✅ BUY:  {best_transfer['In']}")
    print(f"   📈 Gain: +{best_transfer['Gain']:.2f} pts")
    print(f"   💰 Net Cost: £{best_transfer['Cost_Diff']}m")

elif best_transfer:
    print(f"✋ HOLD RECOMMENDED (Gain is only +{max_gain:.2f}):")
    print(f"   The best move is {best_transfer['Out']} -> {best_transfer['In']}")
    print(f"   BUT it does not meet your threshold of +{TRANSFER_THRESHOLD} pts.")
    print(f"   👉 ROLL the transfer. You will have {FREE_TRANSFERS + 1} FTs next week.")
else:
    print("✅ No upgrades found. ROLL transfer.")